# Universal Sentence Encoder Model
This notebook reads in the preprocessed Steam reviews data, fits the USE model to it, and evaluates performance of the trained model.

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
import json
import sklearn

In [7]:
# read in preprocessed and split datasets
train_df = pd.read_csv("data/steam_processed_with_spelling_train.csv")
dev_df = pd.read_csv("data/steam_processed_with_spelling_dev.csv")
test_df = pd.read_csv("data/steam_processed_with_spelling_test.csv")
train_df.head()

,Review,Sentiment
0,an amazing game but your probably better off b...,1.0
1,great game in which you can also shoot some co...,1.0
2,because this is not ripping off agaric,0.0
3,if you are looking for a rift racer this is a ...,1.0
4,played on and fax and rob ram mhz default ultr...,1.0


In [8]:
# replace null reviews with null strings
# not replacing null strings will cause issues with the model
train_df.loc[train_df["Review"].isnull(), 'Review'] = ""
dev_df.loc[dev_df["Review"].isnull(), 'Review'] = ""
test_df.loc[test_df["Review"].isnull(), 'Review'] = ""

## Model Definition and Training

In [9]:
# Training input on the whole training set with no limit on training epochs.
train_input_fn = tf.compat.v1.estimator.inputs.pandas_input_fn(
    train_df, train_df["Sentiment"], num_epochs=None, shuffle=True)

In [10]:
# Prediction on the whole training set.
predict_train_input_fn = tf.compat.v1.estimator.inputs.pandas_input_fn(
    train_df, train_df["Sentiment"], shuffle=False)
# Prediction on the dev set.
predict_dev_input_fn = tf.compat.v1.estimator.inputs.pandas_input_fn(
    dev_df, dev_df["Sentiment"], shuffle=False)
# Prediction on the test set.
predict_test_input_fn = tf.compat.v1.estimator.inputs.pandas_input_fn(
    test_df, test_df["Sentiment"], shuffle=False)

In [11]:
embedded_text_feature_column = hub.text_embedding_column(
    key="Review", 
    module_spec="https://tfhub.dev/google/nnlm-en-dim128/1")

In [13]:
tf.keras.backend.set_floatx('float32')
estimator = tf.estimator.DNNClassifier(
    hidden_units=[500, 100],
    feature_columns=[embedded_text_feature_column],
    n_classes=2,
    optimizer=tf.keras.optimizers.Adagrad(lr=0.003))

INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_model_dir': '/var/folders/j9/9tb4b_ds1r14clxss85c8xq80000gn/T/tmp3oapg69m', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '/var/folders/j9/9tb4b_ds1r14clxss85c8xq80000gn/T/tmp3oapg69m', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [14]:
# train model on data
estimator.train(input_fn=train_input_fn, steps=5000)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
To construct input pipelines, use the `tf.data` module.


...

INFO:tensorflow:loss = 0.29516077, step = 4900 (0.691 sec)


INFO:tensorflow:Saving checkpoints for 5000 into /var/folders/j9/9tb4b_ds1r14clxss85c8xq80000gn/T/tmp3oapg69m/model.ckpt.


INFO:tensorflow:Saving checkpoints for 5000 into /var/folders/j9/9tb4b_ds1r14clxss85c8xq80000gn/T/tmp3oapg69m/model.ckpt.


INFO:tensorflow:Loss for final step: 0.34874964.


INFO:tensorflow:Loss for final step: 0.34874964.


In [15]:
# evaluate on dev data
train_eval_result = estimator.evaluate(input_fn=predict_train_input_fn)
dev_eval_result = estimator.evaluate(input_fn=predict_dev_input_fn)

print("Training set accuracy: {accuracy}".format(**train_eval_result))
print("Dev set accuracy: {accuracy}".format(**dev_eval_result))

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.



If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.84140575, accuracy_baseline = 0.8245806, auc = 0.822683, auc_precision_recall = 0.95474225, average_loss = 0.36252248, global_step = 5000, label/mean = 0.8245806, loss = 0.36253425, precision = 0.8484296, prediction/mean = 0.8297411, recall = 0.98333836


INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.84140575, accuracy_baseline = 0.8245806, auc = 0.822683, auc_precision_recall = 0.95474225, average_loss = 0.36252248, global_step = 5000, label/mean = 0.8245806, loss = 0.36253425, precision = 0.8484296, prediction/mean = 0.8297411, recall = 0.98333836


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: /var/folders/j9/9tb4b_ds1r14clxss85c8xq80000gn/T/tmp3oapg69m/model.ckpt-5000


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: /var/folders/j9/9tb4b_ds1r14clxss85c8xq80000gn/T/tmp3oapg69m/model.ckpt-5000


Training set accuracy: 0.8409846425056458
Dev set accuracy: 0.8414057493209839


## Custom Iterator

In [12]:
#To facilitate scaling the hyper parameter tuning and allow us to try multiple scenarios, we created this custom iterator:

#We start by loading a csv file with all the parameters that we want to test, in this case, 25 different scenarios by tuning optimizer, activation function, dropout, etc.

df = pd.read_csv("search_parameters.csv")
df

,Hidden Units,Learning Rate,Steps,Instances,Optimizer,Activation Function,Batch Normalization,Dropout
0,"500, 100",0.003,5000,"100,000",Adagrad,Relu,None,None
1,"1000, 100",0.003,10000,"100,000",Adagrad,Relu,None,None
2,"1000, 100",0.005,10000,"100,000",Adagrad,Relu,None,None
3,"1000, 100",0.001,10000,"100,000",Adagrad,Relu,None,None
4,"20000, 100",0.001,10000,"100,000",Adagrad,Relu,None,None
5,"20000, 100",0.005,10000,"100,000",Adagrad,Relu,None,None
6,"1000, 100",0.005,10000,"100,000",Adadelta,Relu,None,None
7,"1000, 100",0.005,10000,"100,000",Adam,Relu,None,None
8,"1000, 100",0.005,10000,"100,000",Adamax,Relu,None,None
9,"1000, 100",0.005,10000,"100,000",SGD,Relu,None,None


In [ ]:
train_results=[]
dev_results=[]

#We thn run an iteration per every row contained in the file

for i in range(len(df)):

    
    #We pre-process the input
    
    learning_rate=float(df['Learning Rate'][i])

    activation_functions={'Relu':tf.nn.relu, 'Sigmoid':tf.nn.sigmoid,'Selu':tf.nn.selu,'Softmax':tf.nn.softmax,'Tanh':tf.nn.tanh,'Elu':tf.nn.elu}
    optimizers={'Adagrad':tf.keras.optimizers.Adagrad(lr=learning_rate), 'Adadelta':tf.keras.optimizers.Adadelta(lr=learning_rate), 'Adam':tf.keras.optimizers.Adam(lr=learning_rate), 'Adamax':tf.keras.optimizers.Adamax(lr=learning_rate), 'SGD':tf.keras.optimizers.SGD(lr=learning_rate), 'RMSprop':tf.keras.optimizers.RMSprop(lr=learning_rate), 'Nadam':tf.keras.optimizers.Nadam(lr=learning_rate)}



    hidden_units=[]
    for j in range(len(df['Hidden Units'][i].split(','))):
        hidden_units.append(int(df['Hidden Units'][i].split(',')[j].strip()))

    steps=int(df['Steps'][i])

    optimizer=optimizers[df['Optimizer'][i]]

    activation_fn=activation_functions[df['Activation Function'][i]]

    batch_norm=True
    if(df['Batch Normalization'][i]):
        batch_norm=True

    dropout=None
    if(df['Dropout'][i]!='None'):
        dropout=float(df['Dropout'][i])


    print("Training iteration "+str(i))

    print(optimizer)


    #Then we run the estimator with the parameters
    
    tf.keras.backend.set_floatx('float32')
    estimator = tf.estimator.DNNClassifier(
        hidden_units=hidden_units,
        feature_columns=[embedded_text_feature_column],
        n_classes=2,
        activation_fn=activation_fn,
        batch_norm=batch_norm,
        dropout=dropout,
        optimizer=optimizer)

    estimator.train(input_fn=train_input_fn, steps=steps)
    
    #We then predict the results

    train_eval_result = estimator.evaluate(input_fn=predict_train_input_fn)
    dev_eval_result = estimator.evaluate(input_fn=predict_dev_input_fn)

    print("Training set accuracy: {accuracy}".format(**train_eval_result))
    print("Dev set accuracy: {accuracy}".format(**dev_eval_result))
    

    train_results.append(train_eval_result)
    dev_results.append(dev_eval_result)
    
    

Training iteration 0
INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_model_dir': '/var/folders/w0/zxbb41sj6qq4h6v20cmk37nm0000gq/T/tmp_76dyuog', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '/var/folders/w0/zxbb41sj6qq4h6v20cmk37nm0000gq/T/tmp_76dyuog', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


...

INFO:tensorflow:loss = 0.48390454, step = 3100 (0.672 sec)


INFO:tensorflow:global_step/sec: 125.812


INFO:tensorflow:global_step/sec: 125.812


INFO:tensorflow:loss = 0.3400138, step = 3200 (0.794 sec)


INFO:tensorflow:loss = 0.3400138, step = 3200 (0.794 sec)


## Error Analysis

In [56]:
# get the predicted data from the dev reviews
dev_labels = list(dev_df["Sentiment"])
dev_predictions = list(estimator.predict(input_fn=predict_dev_input_fn))
dev_predictions_classes = [float(prediction["class_ids"][0]) for prediction in dev_predictions]

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.



If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Restoring parameters from /var/folders/j9/9tb4b_ds1r14clxss85c8xq80000gn/T/tmp3oapg69m/model.ckpt-5000


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


In [57]:
# In this particular case, we got 614 true negatives and 16289 true positives
# 2910 false positives and 276 false negatives
confusion_matrix = tf.math.confusion_matrix(dev_labels, dev_predictions_classes)
print(confusion_matrix)

tf.Tensor(
[[  614  2910]
 [  276 16289]], shape=(2, 2), dtype=int32)


In [77]:
# examples of misclassified reviews
misclassified_reviews_indices = (i for i in range(len(dev_labels)) if dev_labels[i] != dev_predictions_classes[i])
dev_df.loc[misclassified_reviews_indices, :].head(10)

,Review,Sentiment
2,this game sucks man i do not know why people l...,0.0
3,kinda hard to enjoy when all you do is lose,0.0
6,early access review,0.0
11,all the men died either during the scavenger h...,1.0
13,,0.0
15,grind grind grind grind grind in this game gai...,0.0
23,the game for me is quite good to play the cons...,1.0
31,early access review,0.0
38,it just crashes,0.0
42,sure it may be entertaining for the first minu...,0.0
